# KNN VS BNN

**Nama  : Rizki Ardian Samudra**

**NIM   : 210411100179**

**Kelas : Penambangan Data-C**

KNN pada dasarnya adalah algoritma supervised machine learning. Dalam bahasa yang lebih sederhana ialah mencari pola dalam data yang ada untuk membuat prediksi. Prediksi tersebut bisa menjadi memprediksi harga mobil atau mengidentifikasi pelanggan mana yang akan merespons dengan baik pada kampanye pemasaran email baru.

Backpropagation Neural Network (jaringan saraf tiruan) pada dasarnya adalah algoritma machine learning yang digunakan untuk melatih model neural network dengan menggunakan data input-output yang telah diketahui. Algoritma backpropagation beroperasi dengan cara mengalirkan input melalui lapisan-lapisan neuron (neural network) untuk menghasilkan output, kemudian menghitung error atau kesalahan prediksi antara output yang dihasilkan oleh model dan output yang sebenarnya.

Mengimport library yang diperlukan

In [ ]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Import data kita

In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/rizkiardian/c27b9d4340fe70046c848bb83d3f6cc5/raw/1aa7222d046761b7b5df703e9f38063ad227aa72/IrisDatasetAsli')

jumlah_baris = df.shape[0]
df.head(jumlah_baris)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


Mengecek nilai yang hilang/kosong pada data

In [ ]:
df_copy = df.copy()
df_copy.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

Mengganti nilai yang mengandung tanda tanya (?) menjadi nilai kosong

In [ ]:
df_copy = df_copy.replace('?',np.nan)

Mengecek lagi berapa nilai yang kosong pada data

In [ ]:
df_copy.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

Setelah itu kita lihat type data pada fitur, jika type datanya bukan numerik/float maka perlu diganti type datanya

In [ ]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


Memisahkan fitur dan target

In [ ]:
# features and target
X = df_copy[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y = df_copy['species']

Melakukan Normalisasi Data pada Fitur dan Target

In [ ]:
# Normalisasi pada X
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Normalisasi pada y
label = LabelEncoder()
y = label.fit_transform(y)

Menyimpan data normalisasi

In [ ]:
# Menyimpan normalisasi ke file
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Menyimpan normalisasi ke file
with open('label.pkl', 'wb') as file:
    pickle.dump(label, file)

Melihat Data yang sudah di Normalisasi

In [ ]:
# Membuat DataFrame dari X dan y menggunakan pandas
liat = pd.DataFrame(X, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
liat['target'] = y

# Menampilkan DataFrame yang digabungkan
liat.head(jumlah_baris)

,sepal_length,sepal_width,petal_length,petal_width,target
0,0.222222,0.625000,0.067797,0.041667,0
1,0.166667,0.416667,0.067797,0.041667,0
2,0.111111,0.500000,0.050847,0.041667,0
3,0.083333,0.458333,0.084746,0.041667,0
4,0.194444,0.666667,0.067797,0.041667,0
...,...,...,...,...,...
145,0.666667,0.416667,0.711864,0.916667,2
146,0.555556,0.208333,0.677966,0.750000,2
147,0.611111,0.416667,0.711864,0.791667,2
148,0.527778,0.583333,0.745763,0.916667,2


Membagi dataset menjadi data training dan data testing (0.8/0.2)

In [ ]:
# Split data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

## 1. Membangun model KNN

In [ ]:
# Membangun model KNN
knn = KNeighborsClassifier(n_neighbors=3)

# Melatih model dengan data latih
knn.fit(X_train, y_train)

# Memprediksi kelas target untuk data uji
y_prediksi_knn = knn.predict(X_test)

# Menghitung akurasi prediksi
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_prediksi_knn)))

Accuracy: 1.00


In [ ]:
# Membuat DataFrame dari X dan y menggunakan pandas
liat_knn = pd.DataFrame(y_test, columns=['y_test'])
liat_knn['y_prediksi_knn'] = y_prediksi_knn

# Menampilkan DataFrame yang digabungkan
liat_knn.head(jumlah_baris)

,y_test,y_prediksi_knn
0,1,1
1,0,0
2,2,2
3,1,1
4,1,1
5,0,0
6,1,1
7,2,2
8,1,1
9,1,1


Menyimpan model KNN

In [ ]:
# Menyimpan model ke file
with open('knn_model.pkl', 'wb') as file:
    pickle.dump(knn, file)

Mencoba mengkategorikan target dari fitur yang diberikan

In [ ]:
# Load model dari file
with open('knn_model.pkl', 'rb') as file:
    knn_loaded = pickle.load(file)

# Load normalisasi dari file
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('label.pkl', 'rb') as file:
    label = pickle.load(file)

# Buat data baru
data_baru = np.array([[5.1, 3.5, 1.4, 0.2]])

# Scaling data baru menggunakan skalar yang telah dilatih sebelumnya
data_baru_scaled = scaler.fit_transform(data_baru)

# Menggunakan model untuk melakukan prediksi pada data baru
y_pred = knn_loaded.predict(data_baru_scaled)
y_pred_scaled = label.inverse_transform(y_pred)

print(y_pred_scaled)

['setosa']


## 2. Membangun model BNN

In [ ]:
# Membangun model BNN
bnn = MLPClassifier(random_state=42, hidden_layer_sizes=(150,100,50),
                        max_iter = 300, activation = 'logistic',
                        solver = 'adam', alpha=0.0001)

# melakukan training pada model 
bnn.fit(X_train, y_train)

# Memprediksi kelas target untuk data uji
y_prediksi_bnn = bnn.predict(X_test)

# Menghitung akurasi prediksi
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_prediksi_bnn)))

Accuracy: 0.97


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# Membuat DataFrame dari X dan y menggunakan pandas
liat_bnn = pd.DataFrame(y_test, columns=['y_test'])
liat_bnn['y_prediksi_bnn'] = y_prediksi_bnn

# Menampilkan DataFrame yang digabungkan
liat_bnn.head(jumlah_baris)

,y_test,y_prediksi_bnn
0,1,1
1,0,0
2,2,2
3,1,1
4,1,1
5,0,0
6,1,1
7,2,2
8,1,2
9,1,1


Menyimpan model KNN

In [ ]:
# Menyimpan model ke file
with open('bnn_model.pkl', 'wb') as file:
    pickle.dump(bnn, file)

Mencoba mengkategorikan target dari fitur yang diberikan

In [ ]:
# Load model dari file
with open('bnn_model.pkl', 'rb') as file:
    bnn_loaded = pickle.load(file)

# Load normalisasi dari file
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('label.pkl', 'rb') as file:
    label = pickle.load(file)

# Buat data baru
data_baru = np.array([[5.1, 3.5, 1.4, 0.2]])

# Scaling data baru menggunakan skalar yang telah dilatih sebelumnya
data_baru_scaled = scaler.fit_transform(data_baru)

# Menggunakan model untuk melakukan prediksi pada data baru
y_pred = bnn_loaded.predict(data_baru_scaled)
y_pred_scaled = label.inverse_transform(y_pred)

print(y_pred_scaled)

['setosa']
